In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('future.no_silent_downcasting', True)

In [32]:
df1=pd.read_excel("data-raw/S1/File 1-1.xlsx")
df2=pd.read_csv("data-raw/S2/File 2-1.csv")
df3=pd.read_excel("data-raw/S3/File 3-1.xlsx")

In [33]:
df1.info()
df2.info()
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Activity Name         50 non-null     object 
 1   Sector-Category       50 non-null     object 
 2   Unit                  50 non-null     object 
 3   Emmision (kgCO2e)     50 non-null     float64
 4   kgCO2                 50 non-null     float64
 5   kgCH4                 50 non-null     float64
 6   kgN2O                 50 non-null     object 
 7   Assesment Report      50 non-null     object 
 8   Scope                 50 non-null     int64  
 9   Life Cylce Assesment  50 non-null     object 
 10  Validity Year         50 non-null     int64  
 11  Validity Region       50 non-null     object 
 12  Source                50 non-null     object 
dtypes: float64(3), int64(2), object(8)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29


In [35]:
df1.sample(1)

,Activity Name,Sector-Category,Unit,Emmision (kgCO2e),kgCO2,kgCH4,kgN2O,Assesment Report,Scope,Life Cylce Assesment,Validity Year,Validity Region,Source
29,North Europe to-from North America WC - Contai...,Transport/Sea Freight,TEU-km,16.222096,4.205493,0.445059,not-supplied,AR6,2,primary_production-transport-use,2023,GLOBAL,S1


## Observations

- Different file_type -> data comming from different sources (S1,S2,S3)
- Different number of columns  -> S1 has 12 others 13 -> Sector-Category (S1) vs Sector and Category (S2, S3)
- Different naming of columns:
    - Emmision (kgCO2e) (S1) vs kgCO2e (S2, S3)
    - LCA (S3) vs Life Cylce Assesment (S1, S2)
    - Year Valid From (S3) vs Validity Year (S2)   -> We assume they mean the same
    - Region (S3) vs Validity Region (S1, S2)

- Emissions have missing values and values like  'not-supplied', 'not_supplied' -> we make them np.nan
- some categorical columns have missing values like: 'Unknown', 'unknown' -> we make them None/ NULL
- Data types are different -> best data_type for each column?


- activity_name: can have duplicated values -> no primary key
- sector, category, unit, scope, lca: can have "duplicates" like Transport vs transport... activity_name could also have it but maybe not worth changing


## Questions/Asumptions 

- We need more domain knowledge to understand relations between columns
- Who is going to use the database? for what?
- What is the priority WRITE vs READ?  (Norm vs Denorm)
- How specific should region be? continent, contry, state....
- For the units? probably could include conversion factors
- How much do we know from the source? AWS/GCP, API, email, file type, contact
- Activity looks like it has a direct relation with sector and category
- Simple star schema no need to overcomplicate
- Fact table from emissions (numerical). Dimension tables: activities, region, sources, units...

# Review Process

Create specific tables for the reviewing process in the schema.

- Tables for status, users, change_requests
- Users can have different roles (editor, reviewer, admin....)

## Process flow

By default all entries have the status 'pending_review'

1. User identifies a change/error/issue with an emission factor
2. User can submit a request with:
    - emission_factor_id
    - proposed_changes in JSON form
    - requested_by_id
    - requested_at: This one is set automatically  
3. Another user reviews the request
    - reviewed_by_id
    - adds review_comments
    - changes to table according to JSON
    - changes status to 'aprroved' or 'rejected' 



  





In [2]:
paths = ["data-raw/S1/File 1-1.xlsx","data-raw/S2/File 2-1.csv", "data-raw/S3/File 3-1.xlsx"]
# Standardize column names
STANDARD_COLUMNS = ['activity_name', 'sector', 'category', 'unit', 'kgco2e', 'kgco2', 'kgch4', 'kgn2o',
       'assesment_report', 'scope', 'lca', 'validity_year',
       'region', 'source', 'file_type']

all_data = []
for file_path in paths:
    extension = file_path.split(".")[1]
    source = file_path.split("/")[1]
    if extension == "csv":
        df = pd.read_csv(file_path)
    else:
        df = pd.read_excel(file_path)
    
    # Split Sector-Category
    if source == "S1":   
        df[['Sector', 'Category']] = df['Sector-Category'].str.split('/', expand=True)
        df.drop(['Sector-Category'], axis=1, inplace=True)

    # Standarize columns
    df.columns = df.columns.str.lower().str.rstrip().str.replace(' ', '_').str.replace('-', '_')
    df = df.rename(columns={
        'emmision_(kgco2e)': 'kgco2e',
        'life_cylce_assesment': 'lca',
        'year_valid_from': 'validity_year',
        'validity_region': 'region'
    })
    # add source file and file_type
    df['source'] = source
    df[ 'file_type'] = extension

    # clean strings to avoid duplicates like Transport and transport
    for column in ['sector', 'category', 'unit','scope', 'lca']:      # Open question: activity_name ????
        df[column] = df[column].apply(lambda x: str(x).lower().rstrip())
    
    # clean missing values
    df.replace(['unknown', 'Unknown'], None, inplace=True)
    df.replace(["not_supplied", "not-supplied"], np.nan, inplace=True)

    df = df.reindex(columns=STANDARD_COLUMNS)
    all_data.append(df)
    
combined_df = pd.concat(all_data)

/tmp/ipykernel_67281/785940117.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_data)


In [5]:
print(combined_df.columns)
combined_df.sample(2)

Index(['activity_name', 'sector', 'category', 'unit', 'kgco2e', 'kgco2',
       'kgch4', 'kgn2o', 'assesment_report', 'scope', 'lca', 'validity_year',
       'region', 'source', 'file_type'],
      dtype='object')


,activity_name,sector,category,unit,kgco2e,kgco2,kgch4,kgn2o,assesment_report,scope,lca,validity_year,region,source,file_type
39,North Europe to-from Mediterranean/Black Sea -...,transport,sea freight,teu-km,18.333037,8.738967,0.355336,NaN,AR6,2,primary_production-transport,2023,GLOBAL,S1,xlsx
25,Articulated truck 44-60t - Container - Diesel,transport,road freight,tonne-km,12.391327,2.071839,0.382203,NaN,AR6,2,use,2023,GLOBAL,S1,xlsx


In [12]:
combined_df["assesment_report"]
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [89]:
df1=all_data[0]
#df1.info()
df2=all_data[1]
#df2.info()
df3=all_data[2]
#df3.info()
display(df1.sample())
display(df2.sample())
display(df3.sample())

,activity_name,sector,category,unit,kgco2e,kgco2,kgch4,kgn2o,assesment_report,scope,lca,validity_year,region,source
17,Other liquids tankers <999 dwt - MDO,Transport,Sea Freight,tonne-km,30.988433,4.01066,0.999177,NaN,AR6,2,NaN,2023,GLOBAL,S1


,activity_name,sector,category,unit,kgco2e,kgco2,kgch4,kgn2o,assesment_report,scope,lca,validity_year,region,source
6,Sub-bituminous coal - industrial,energy,fuel,Kg,2.775364,2.670836,NaN,0.000418,AR4,2,NaN,2023,GB,S2


,activity_name,sector,category,unit,kgco2e,kgco2,kgch4,kgn2o,assesment_report,scope,lca,validity_year,region,source
21,PP waste disposal (incl. forming) - open-loop,waste,Plastic Waste,tonne,141.393414,9.387046,4.889052,0.000008,AR4,None,end_of_life,2021,US-TX,S3


In [7]:
for x in ["scope","assesment_report", 'source']:
    print(combined_df[x].value_counts())
    print("########")

scope
2    62
1    11
3     7
Name: count, dtype: int64
########
assesment_report
AR4    80
AR6    50
Name: count, dtype: int64
########
source
S1    50
S3    50
S2    30
Name: count, dtype: int64
########


In [20]:
df_copy = combined_df.copy()
for column in ['activity_name']:
        df_copy[column] = df_copy[column].apply(lambda x: str(x).lower().rstrip())

df_copy.activity_name.value_counts()
#combined_df.lca.value_counts()

activity_name
hotel stay (3 star)                                                                                 3
liquefied petroleum gas lpg - transport                                                             2
liquefied gas tanker 50000-99999 cbm - mdo                                                          2
container ship - reefer (refrigerated)                                                              1
europe (north & med) to- from oceania (via suez / via panama) - container ship (dry)                1
freight logistics - transshipment - mixed                                                           1
asia to-from africa - container ship - reefer (refrigerated)                                        1
rigid truck 7.5-12t - average/mixed load - diesel                                                   1
rail freight (electric traction) - trailer only on train - distribution losses                      1
north america ec/gulf/wc to-from middle east/india - container ship 

In [17]:
combined_df[["activity_name"]]

,activity_name
0,Europe (North & Med) to- from Oceania (via Sue...
1,Rigid truck 3.5-7.5t - Average/mixed load - Di...
2,Container ship - reefer (refrigerated)
3,Freight logistics - transshipment - mixed
4,Asia to-from North America WC - Container ship...
5,Air freight (<1500 km) - freighter - RP1678
6,Asia to-from Africa - Container ship - reefer ...
7,Rigid truck 7.5-12t - Average/mixed load - Diesel
8,Rail freight (electric traction) - trailer onl...
9,North America EC/Gulf/WC to-from Middle East/I...


In [6]:

combined_df.activity_name.value_counts()
combined_df[combined_df["activity_name"].isin(["Hotel stay (3 star)", "Liquefied petroleum gas LPG - transport", "Liquefied gas tanker 50000-99999 cbm - MDO"])]

,activity_name,sector,category,unit,kgco2e,kgco2,kgch4,kgn2o,assesment_report,scope,lca,validity_year,region,source
35,Liquefied gas tanker 50000-99999 cbm - MDO,Transport,Sea Freight,tonne-km,23.390201,4.510120,0.699262,NaN,AR6,2,NaN,2023,GLOBAL,S1
44,Liquefied gas tanker 50000-99999 cbm - MDO,Transport,Sea Freight,tonne-km,16.304026,4.385202,0.441438,NaN,AR6,2,NaN,2023,GLOBAL,S1
3,Liquefied petroleum gas LPG - transport,energy,fuel,L,2.859979,2.766725,NaN,0.000373,AR4,1,NaN,2023,US,S2
25,Liquefied petroleum gas LPG - transport,energy,fuel,L,2.094755,2.094755,NaN,NaN,AR4,3,NaN,2020,DE,S2
33,Hotel stay (3 star),restaurants and accommodation,Accommodation,room-night,81.138327,2.780048,2.902158,NaN,AR4,None,None,2022,US-MN,S3
34,Hotel stay (3 star),restaurants and accommodation,Accommodation,room-night,34.752395,8.883738,0.958047,0.000006,AR4,None,None,2022,US-TX,S3
48,Hotel stay (3 star),restaurants and accommodation,Accommodation,room-night,102.946649,2.433086,3.722682,0.000005,AR4,None,None,2022,US-TX,S3


In [107]:
combined_df[combined_df["activity_name"].isin(["Hotel stay (3 star)", "Liquefied petroleum gas LPG - transport"])].to_csv("test1.csv", index=False)

In [109]:
combined_df.sample(5).to_csv("test2.csv", index=False)

In [27]:
def clean_s1(file_name):
    """Clean and standardize file 1"""
    df = pd.read_excel(f"data-raw/S1/{file_name}")
    
    # Split Sector-Category
    df[['Sector', 'Category']] = df['Sector-Category'].str.split('/', expand=True)
    
    # Standardize column names
    df = df.rename(columns={
        'Emmision (kgCO2e)': 'kgCO2e',
        'Life Cylce Assesment': 'LCA',
        'Validity Year': 'Validity_Year',
        'Validity Region': 'Region'
    })
    
    # Handle 'not-supplied' values
    df = df.replace('not-supplied', np.nan)
    
    return df

def clean_s2(file_name):
    """Clean and standardize file 2"""
    df = pd.read_csv(f"data-raw/S2/{file_name}")
    
    # Rename columns
    df = df.rename(columns={
        'Life Cylce Assesment': 'LCA',
        'Validity Year': 'Validity_Year',
        'Validity Region': 'Region'
    })
    
    return df

def clean_s3(file_path):
    """Clean and standardize file 3"""
    df = pd.read_excel(f"data-raw/S3/{file_name}")
    
    # Rename columns
    df = df.rename(columns={
        'Year Valid From': 'Validity_Year',
        'Region': 'Region'
    })
    
    # Handle 'Unknown' scope
    df['Scope'] = df['Scope'].replace('Unknown', None)
    
    return df

file1 12 ['Activity Name', 'Sector-Category', 'Unit ', 'Emmision (kgCO2e)', 'kgCO2', 'kgCH4', 'kgN2O', 'Assesment Report', 'Scope', 'Life Cylce Assesment', 'Validity Year', 'Validity Region']
file2 13 ['Activity Name', 'Sector', 'Category', 'Unit ', 'kgCO2e', 'kgCO2', 'kgCH4', 'kgN2O', 'Assesment Report', 'Scope', 'Life Cylce Assesment', 'Validity Year', 'Validity Region']
file3 13 ['Activity Name', 'Unit ', 'kgCO2e', 'kgCO2', 'kgCH4', 'kgN2O', 'Assesment Report', 'Scope', 'LCA', 'Sector', 'Category', 'Year Valid From', 'Region']


,Activity Name,Sector-Category,Unit,Emmision (kgCO2e),kgCO2,kgCH4,kgN2O,Assesment Report,Scope,Life Cylce Assesment,Validity Year,Validity Region
40,Chemical tanker >40000 dwt - HFO,Transport/Sea Freight,tonne-km,6.663576,4.81487,0.068471,not-supplied,AR6,2,primary_production-transport-use,2023,GLOBAL


In [ ]:
CREATE TABLE Emissions (
    emission_id SERIAL PRIMARY KEY,
    activity_id INT REFERENCES Activities(activity_id),
    kgco2e FLOAT,
    kgco2 FLOAT,
    kgch4 FLOAT,
    kgn2o FLOAT,
    assessment_report VARCHAR(50),
    scope VARCHAR(50),
    lca VARCHAR(50),
    validity_year INT,
    region VARCHAR(100),
    source VARCHAR(50)
);

In [ ]:
-- Main emission factors table
CREATE TABLE emission_factors (
    emission_factor_id SERIAL PRIMARY KEY,
    activity_id INTEGER NOT NULL REFERENCES activities(activity_id),
    unit_id INTEGER NOT NULL REFERENCES units(unit_id),
    region_id INTEGER NOT NULL REFERENCES regions(region_id),
    source_id INTEGER NOT NULL REFERENCES sources(source_id),
    scope scope_type,
    assessment_report assessment_report_type,
    validity_year INTEGER,
    lca BOOLEAN,
    kgco2e DECIMAL(20,10),
    kgco2 DECIMAL(20,10),
    kgch4 DECIMAL(20,10),
    kgn2o DECIMAL(20,10),

    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    
    CONSTRAINT emission_factors_validity_year_check CHECK (validity_year > 1900),
    CONSTRAINT emission_factors_date_check CHECK (valid_until IS NULL OR valid_until > valid_from)
);